In [1]:
# show image of a QR code 
# a QR code is generated and IP of device displayed on the picture. 
# The QR requests an authorization token from spotify API and the link includes values
#The URL for the QR uses spotify oAuth to create a permanent refresh token
# such as client ID, redirect uri, and scope and 
# in the future maybe store client info in encrypted files...

def generate_AUTH_QR():
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    import qrcode
    import socket
    from PIL import Image, ImageDraw, ImageFont
    #import Gameplay.ttf
    # Define the URL to be encoded in the QR code
    CLIENT_ID = "57bab06475f04739ae8f759a21bc28f7"
    CLIENT_SECRET = "4b8159fffce14f2eb28d0a5e6e35b7c6"
    REDIRECT_URI = "https://authorize.letsmakemusic.site/"
    SCOPE = "playlist-read-private"
    #REDIRECT_URI = "http://musicman.epizy.com/callback?code="
    #SCOPE = "playlist-read-private"
    
    # Create an instance of the SpotifyOAuth object
    sp_oauth = SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE
            )

    # Obtain a temporary access token
    authorization_url = sp_oauth.get_authorize_url()
    #authorization_url = f"https://accounts.spotify.com/authorize?client_id={CLIENT_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope={SCOPE}"
    
    # Get the device IP address
    ip_address = socket.gethostbyname(socket.gethostname())
    
    print(authorization_url)
    print(ip_address)
    
    # Create the QR code image
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(authorization_url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    draw_txt = ImageDraw.Draw(img)
    # Add the device IP address to the 
    font_size = 40
    font = ImageFont.truetype("Gameplay.ttf", size=font_size)
    img_width, img_height = img.size
    ip_text = "Device IP: " + ip_address
    txt_width, txt_height = draw_txt.textsize(ip_text, font=font)
    #ip_text_width, ip_text_height = font.getsize(ip_text)
    img_with_ip = img.resize((img_width, img_height + txt_height + 10))
    qrcode_draw = ImageDraw.Draw(img_with_ip)
    qrcode_draw.text(((img_width - txt_width) / 2, img_height - txt_height + 45), ip_text, font=font, fill="black")

    # Show the final image
    img_with_ip.show()


In [13]:
#app4 working in console
#Working IP Server for grabbing URL
# This code starts a flask server over port 5000 on your device IP
# Once a token is succesfully recieved, it is saved into a text file and stored on device
# The server then shuts down

def start_oauth_token_server():
    from flask import Flask, request, jsonify
    from flask_cors import CORS
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    from flask_wtf.csrf import CSRFProtect
    
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'your-secret-key'
    csrf = CSRFProtect(app)
    #CORS(app, resources={r"/process_auth_code": {"origins": "http://musicman.epizy.com"}})
    CORS(app, resources={r"/process_auth_code": {"origins": ["https://authorize.letsmakemusic.site"]}})

    def shutdown_server():
        func = request.environ.get('werkzeug.server.shutdown')
        if func is None:
            raise RuntimeError('Not running with the Werkzeug Server')
        func()

    @app.route('/process_auth_code', methods=['OPTIONS', 'POST'])
    def process_auth_code():
        auth_code = ""
        file_name = "refresh_token_auth_spotify_TEST1.txt"
        #certificate = 
        #key = 
        #file_path = "C:/User/alexk/Desktop/Alex Music Stuff/Spotify_ML_Proj/" + file_name
        if request.method == 'OPTIONS':
            # Set CORS headers for preflight request
            headers = {
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST',
                'Access-Control-Allow-Headers': 'Content-Type',
                'Access-Control-Max-Age': '3600'
            }
            return ('', 200, headers)

        try:
            auth_code = request.json['authorization_code']
            print(f"Received authorization code:", auth_code)
            token_info = sp_oauth.get_access_token(code)

            # Exchange the access token for a refresh token
            #sp = spotipy.Spotify(auth=token_info['access_token'])
            refresh_token = token_info['refresh_token']

            with open(file_name, "w") as f:
                f.write(auth_code)
            shutdown_server()
            #return auth_code
            return jsonify({'message': 'Authorization code received successfully!'})
        except Exception as e:
            print(f"Error receiving authorization code: {e}", file=sys.stderr)
            return jsonify({'message': 'Authorization code not received!'}), 400

    #if __name__ == '__main__':
    try:
        # To run on http (was working before...)
        #app.run(host='0.0.0.0', port=5000)
        # To run on https: (not working)
        app.run(host='0.0.0.0', port=5000, ssl_context=("fullchain2.pem", "privkey2.pem"))
    except Exception as e:
        print(f"Error running Flask app: {e}", file=sys.stderr)

    print("done")
    #return authcode1
## possible error was a different old auth code was being passed to the finally statement...

In [7]:
generate_AUTH_QR()

https://accounts.spotify.com/authorize?client_id=57bab06475f04739ae8f759a21bc28f7&response_type=code&redirect_uri=https%3A%2F%2Fauthorize.letsmakemusic.site%2F&scope=playlist-read-private
10.8.21.132


C:\Users\alexk\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


In [17]:
start_oauth_token_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Received authorization code
done


In [8]:
def refresh_auth():
    file_name = "refresh_token_auth_spotify_TEST1.txt"
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    
    try: 
        # try to open file first, if no error, check token
        try:
            with open(file_name, "r") as f:
                token = f.read()
        except:
            print("Invalid Refresh token file. Please Authorize Spotify now.")
        
        else:
            refresh_token_auth = sp_oauth.refresh_access_token(refresh_token)

    except:
        print("Authorization failed. Please Authorize Spotify now.")
    
    # if token is valid, acquire access token 
    else:
        access_token = refresh_token_auth['access_token']
        sp = spotipy.Spotify(auth=access_token)
        return sp



In [9]:
## this code is not yet tested 2/26/23 - 3:51 am 
## This code will use the sp module taken as an input and return a datafram of the songs in a playlist
# The code gives the user a choice to pick which playlists to access leading to where the target is stored
# The data frame should include all necesary information that the ML model will need to identify
# the songs for training on and analyzing


def get_target_playlist(sp):
    
    # Get user's playlists
    playlists = []
    while True:
        playlist_choice = input("Enter the number corresponding to the type of playlist you want to use:\n1. Private\n2. Public\n3. Personal\n")
        try:
            playlist_choice = int(playlist_choice)
            if playlist_choice < 1 or playlist_choice > 3:
                raise ValueError
            break
        except ValueError:
            print("Invalid input. Please enter a valid playlist number.")

    if playlist_choice == 1:
        playlists = sp.current_user_playlists(limit=50, offset=0, market=None)
    elif playlist_choice == 2:
        playlists = sp.current_user_playlists(limit=50, offset=0, market=None, public=True)
    elif playlist_choice == 3:
        playlists = sp.current_user_playlists(limit=50, offset=0, market=None, me=True)

    # Print playlist names and let user select one
    print("Your playlists:")
    for i, playlist in enumerate(playlists['items']):
        print(f"{i+1}. {playlist['name']}")

    while True:
        playlist_choice = input("Enter the number of the playlist you want to use: ")
        try:
            playlist_choice = int(playlist_choice)
            if playlist_choice < 1 or playlist_choice > len(playlists['items']):
                raise ValueError
            break
        except ValueError:
            print("Invalid input. Please enter a valid playlist number.")

    # Get the selected playlist's tracks
    playlist_id = playlists['items'][playlist_choice-1]['id']
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    # Create a dataframe with the playlist tracks
    df = pd.DataFrame()
    for track in tracks:
        track_data = track['track']
        track_info = {
            'name': track_data['name'],
            'artist': track_data['artists'][0]['name'],
            'album': track_data['album']['name'],
            'release_date': track_data['album']['release_date'],
            'length': track_data['duration_ms']/1000
        }
        df = df.append(track_info, ignore_index=True)

    # Print the playlist tracks dataframe
    print(df)
       
    return df


In [ ]:
## Main script

def main():
    
    import spotipy 
    from spotipy.oauth2 import SpotifyOAuth

    while True:
        try:
            sp = refresh_auth()
        except:

            re_auth = input("Would you like to reauthorize Spotify now? (Y/n): ")
            if re_auth == "y" or re_auth == "Y":
                generate_AUTH_QR()
                server_test_1()
                #start_oauth_token_server()
                #start_oauth_token_server_2()
                continue
            else:
                print("Ending now, thank you.")
                break
        break
    
    try:
        # below is some test code for verifying the refresh token is valid by listing playlists
        playlists = sp.current_user_playlists()
        for playlist in playlists['items']:
            print(playlist['name'])
    except:
        print("U prob broke something bro...")
    
if __name__ == '__main__':
    main()


Would you like to reauthorize Spotify now? (Y/n): y
https://accounts.spotify.com/authorize?client_id=57bab06475f04739ae8f759a21bc28f7&response_type=code&redirect_uri=https%3A%2F%2Fauthorize.letsmakemusic.site%2F&scope=playlist-read-private
10.8.21.132


C:\Users\alexk\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on https://0.0.0.0:5000/ (Press CTRL+C to quit)


In [20]:
#app4 working in console
#Working IP Server for grabbing URL
# This code starts a flask server over port 5000 on your device IP
# Once a token is succesfully recieved, it is saved into a text file and stored on device
# The server then shuts down

def start_oauth_token_server_2():
    from flask import Flask, request, jsonify
    from flask_cors import CORS
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth

    app = Flask(__name__)
    #CORS(app, resources={r"/process_auth_code": {"origins": "http://musicman.epizy.com"}})
    CORS(app, resources={r"/process_auth_code": {"origins": ["http://musicman.epizy.com", "https://musicman.epizy.com"]}})
    app.config['TIMEOUT'] = 120  # 2 minutes
    
    def shutdown_server():
        func = request.environ.get('werkzeug.server.shutdown')
        if func is None:
            raise RuntimeError('Not running with the Werkzeug Server')
        func()

    @app.route('/process_auth_code', methods=['OPTIONS', 'POST'])
    def process_auth_code():
        auth_code = ""
        file_name = "refresh_token_auth_spotify_TEST1.txt"
        #certificate = 
        #key = 
        #file_path = "C:/User/alexk/Desktop/Alex Music Stuff/Spotify_ML_Proj/" + file_name
        if request.method == 'OPTIONS':
            # Set CORS headers for preflight request
            headers = {
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, OPTIONS', # also tried with just post
                'Access-Control-Allow-Headers': 'Content-Type',
                'Access-Control-Max-Age': '3600'
            }
            return ('', 200, headers)

        try:
            auth_code = request.json['authorization_code']
            print(f"Received authorization code:", auth_code)
            token_info = sp_oauth.get_access_token(code)

            # Exchange the access token for a refresh token
            #sp = spotipy.Spotify(auth=token_info['access_token'])
            refresh_token = token_info['refresh_token']

            with open(file_name, "w") as f:
                f.write(auth_code)
            shutdown_server()
            #return auth_code
            return jsonify({'message': 'Authorization code received successfully!'})
        except Exception as e:
            print(f"Error receiving authorization code: {e}", file=sys.stderr)
            return jsonify({'message': 'Authorization code not received!'}), 400

    #if __name__ == '__main__':
    try:
        start_time = time.monotonic()
        while True:
            if time.monotonic() - start_time > app.config['TIMEOUT']:
                raise TimeoutError('Flask app timeout')
            # To run on http (was working before...)
            #app.run(host='0.0.0.0', port=5000)
            # To run on https: (not working)
            app.run(host='0.0.0.0', port=5000, ssl_context=("cert1.pem", "privkey1.pem"))
            time.sleep(1)
    except Exception as e:
        print(f"Error running Flask app: {e}", file=sys.stderr)

    print("done")
    #return authcode1
## possible error was a different old auth code was being passed to the finally statement...

In [7]:
def generate_flask_key():
    import os
    # Generate a new secret key
    secret_key = os.urandom(24)
    # Save the secret key to a file
    with open('secret_key.txt', 'wb') as f:
        f.write(secret_key)
    print("Key generated succesfully.")


def server_test_1():
    import os
    from flask import Flask, request, jsonify
    from flask_cors import CORS
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    from flask_wtf.csrf import CSRFProtect

    app = Flask(__name__)
    # Load the secret key from file
    with open('secret_key.txt', 'rb') as f:
        app.secret_key = f.read()
    
    csrf = CSRFProtect(app)
    CORS(app, resources={r"/process_auth_code": {"origins": ["https://authorize.letsmakemusic.site"]}})

    def shutdown_server():
        func = request.environ.get('werkzeug.server.shutdown')
        if func is None:
            raise RuntimeError('Not running with the Werkzeug Server')
        func()

    @app.route('/process_auth_code', methods=['OPTIONS', 'POST'])
    def process_auth_code():
        auth_code = ""
        file_name = "refresh_token_auth_spotify_TEST1.txt"
        if request.method == 'OPTIONS':
            # Set CORS headers for preflight request
            headers = {
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST',
                'Access-Control-Allow-Headers': 'Content-Type',
                'Access-Control-Max-Age': '3600'
            }
            return ('', 200, headers)

        try:
            auth_code = request.json['authorization_code']
            print(f"Received authorization code:", auth_code)
            token_info = sp_oauth.get_access_token(code)

            # Exchange the access token for a refresh token
            refresh_token = token_info['refresh_token']

            with open(file_name, "w") as f:
                f.write(auth_code)
            shutdown_server()
            return jsonify({'message': 'Authorization code received successfully!'})
        except Exception as e:
            print(f"Error receiving authorization code: {e}", file=sys.stderr)
            return jsonify({'message': 'Authorization code not received!'}), 400

    if __name__ == '__main__':
        # Get the secret key from environment variable
        #secret_key = os.environ.get('FLASK_SECRET_KEY')
        #if not secret_key:
        #    raise ValueError('FLASK_SECRET_KEY environment variable is not set')
        #app.secret_key = secret_key

        try:
            # To run on http (was working before...)
            #app.run(host='0.0.0.0', port=5000)
            # To run on https: (not working)
            app.run(host='0.0.0.0', port=5000, ssl_context=("fullchain2.pem", "privkey2.pem"))
        except Exception as e:
            print(f"Error running Flask app: {e}", file=sys.stderr)

        print("done")


In [5]:
generate_flask_key()

Key generated succesfully.
